# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は **ID 基準（class）**、表示は仕様どおり

## 1) 問題

* `Courses` から **受講生が5人以上いる class を求める**
* 入力: `Courses(student varchar, class varchar)` （主キー: `(student, class)`）
* 出力: `class` のみ（重複なし、順序任意）

## 2) 最適解（単一クエリ）

> PostgreSQL では **ウィンドウ集計**で人数を付与し、閾値で抽出して最終投影。

```sql
WITH win AS (
  SELECT
    class,
    COUNT(*) OVER (PARTITION BY class) AS cnt
  FROM Courses
)
SELECT DISTINCT
  class
FROM win
WHERE cnt >= 5;

Runtime 265 ms
Beats 95.66%

```

### 代替（シンプル集約で十分な場合）

> 行数が多くてもこの形が最短経路です。実務ではまずこちらを使います。

```sql
SELECT
  class
FROM Courses
GROUP BY class
HAVING COUNT(*) >= 5;

Runtime 275 ms
Beats 77.84%

```

## 3) 要点解説

* 主キー `(student, class)` により **同一学生の同一クラス重複は存在しない**ため、単純 `COUNT(*)` で人数カウント可
* ウィンドウ版は「行に人数を付けてから DISTINCT」で表現力が高い
  一方、要件が「グループの閾値判定」だけなら **`GROUP BY ... HAVING`** が最も簡潔で計画も安定しがち
* 物理設計のヒント

  * 頻繁にこの集約を行うなら **`CREATE INDEX ON Courses(class);`** を検討（グループ化やスキャンが有利）

## 4) 計算量（概算）

* ウィンドウ処理: **O(Σ n_g log n_g)**（パーティション内の並び替えコストを含む実装が多い）
* 集約（代替案）: **O(N)** 近似（ハッシュ集約が選ばれる前提）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 テーブル Courses]
  B[前処理 ウィンドウで人数付与]
  C[条件 cnt >= 5]
  D[出力 class 列のみ]
  A --> B
  B --> C
  C --> D
```
